<a href="https://colab.research.google.com/github/Artmagic/GP2_team34/blob/dev/%D0%9F%D0%B0%D1%81%D1%81%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%BF%D0%BE%D1%82%D0%BE%D0%BA_%D0%BF%D0%BE_%D1%81%D1%82%D0%B0%D0%BD%D1%86%D0%B8%D1%8F%D0%BC_%D0%BC%D0%B5%D1%82%D1%80%D0%BE_upd_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import requests
import pandas as pd

url = "https://apidata.mos.ru/v1/datasets"

response = requests.get(url, params={"$top": 2000, "api_key": "a062e7e5-7df3-470f-8751-1b48ffce3931"})
data = response.json()

items = pd.DataFrame(data["Items"])
items.head()

,CategoryId,PublishDate,Keywords,IdentificationNumber,ContainsGeodata,IsSeasonal,IsNew,VersionNumber,Caption,ContainsAccEnvData,IsForeign,IsArchive,SefUrl,Id,ReleaseNumber,DepartmentId,Season,FullDescription,LastUpdateDate
0,107,2014-05-13,"библиотеки, выставка, выставочные залы, дворец...",7702155262-culturehouse,True,False,False,3,Дома культуры и клубы,True,False,False,None,493,1094,1008,0,<p>Набор данных содержит информацию о домах ку...,2025-10-30
1,111,2014-04-10,"библиотеки, выставочные залы, дома культуры и ...",7702155262-MovieTheaters,True,False,False,3,Кинотеатры,True,False,False,None,495,58,1008,0,"<p style=""margin-left:0cm; margin-right:0cm"">Н...",2025-10-30
2,111,2014-01-15,"аттракцион, аттракционы",7709083955-attractions,True,False,False,7,Аттракционы в парках и на площадках,False,False,False,None,498,555,1214,0,<!--[if gte mso 9]><xml>\n <o:OfficeDocumentSe...,2025-11-06
3,112,2014-04-03,"государственные учреждения, государственное уч...",7701236617-DZKHIBgos,True,False,False,3,"Организации, находящиеся в ведомственном подчи...",False,False,False,None,499,26,1006,0,"<p><span style=""font-family:Tahoma; font-size:...",2024-12-06
4,112,2014-04-03,"инженерные службы, инженерная служба, службы, ...",7701236617-Regionalengineeringservices,True,False,False,3,Государственные казенные учреждения «Инженерны...,True,False,True,None,500,30,1006,0,"<p style=""margin-bottom:0cm;margin-bottom:.000...",2022-01-21


In [58]:
items[items["Caption"].str.contains("поток", case=False, na=False)][["Id", "Caption"]]

,Id,Caption
1116,62521,Месячный пассажиропоток по всем видам обществе...
1117,62523,Годовой пассажиропоток по всем видам обществен...
1129,62743,Пассажиропоток по станциям Московского метропо...


In [59]:
import pandas as pd
import time

base_url = f"https://apidata.mos.ru/v1/datasets/62743/rows"
page_size = 1000
skip = 0
all_records = []

while True:
    params = {
        "api_key": "a062e7e5-7df3-470f-8751-1b48ffce3931",
        "$top": page_size,
        "$skip": skip
    }
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"Ошибка {response.status_code} при skip={skip}")
        break

    batch = response.json()
    if not batch:
        break

    # Извлекаем "Cells"
    all_records.extend([b["Cells"] for b in batch])

    skip += page_size

    # Небольшая пауза, чтобы не перегружать API
    time.sleep(0.5)

df = pd.DataFrame(all_records)
print("Итоговый размер:", df.shape)


Итоговый размер: (5533, 7)


In [60]:
df

,NameOfStation,Line,Year,Quarter,IncomingPassengers,OutgoingPassengers,global_id
0,Новаторская,Троицкая линия,2025,III квартал,1167230,1181704,2822445543
1,Университет Дружбы Народов,Троицкая линия,2025,III квартал,887396,877567,2822436510
2,Генерала Тюленева,Троицкая линия,2025,III квартал,473206,474053,2822436511
3,Тютчевская,Троицкая линия,2025,III квартал,310532,314382,2822436512
4,Коммунарка,Троицкая линия,2025,III квартал,517226,523637,2822436515
...,...,...,...,...,...,...,...
5528,Деловой центр,Большая кольцевая линия,2021,I квартал,1062070,1053347,1139050976
5529,Хорошёвская,Большая кольцевая линия,2021,I квартал,687219,687558,1139049567
5530,Петровский парк,Большая кольцевая линия,2021,I квартал,752848,778402,1138976383
5531,ЦСКА,Большая кольцевая линия,2021,I квартал,2337493,2374510,1138976435


In [61]:
station_year_passenger_flow = df.groupby(['NameOfStation', 'Line', 'Year'])[['IncomingPassengers', 'OutgoingPassengers']].sum().reset_index()
station_year_passenger_flow

,NameOfStation,Line,Year,IncomingPassengers,OutgoingPassengers
0,Авиамоторная,Большая кольцевая линия,2023,4081013,4213733
1,Авиамоторная,Большая кольцевая линия,2024,4384158,4845349
2,Авиамоторная,Большая кольцевая линия,2025,3854426,4165770
3,Авиамоторная,Калининская линия,2021,12933490,13555698
4,Авиамоторная,Калининская линия,2022,14050604,14720309
...,...,...,...,...,...
1483,улица Дмитриевского,Некрасовская линия,2021,2896953,2819403
1484,улица Дмитриевского,Некрасовская линия,2022,3142157,3055071
1485,улица Дмитриевского,Некрасовская линия,2023,3390407,3382136
1486,улица Дмитриевского,Некрасовская линия,2024,3421243,3421946


In [62]:
station_year_passenger_flow = station_year_passenger_flow[station_year_passenger_flow.Year == 2024]
station_year_passenger_flow

,NameOfStation,Line,Year,IncomingPassengers,OutgoingPassengers
1,Авиамоторная,Большая кольцевая линия,2024,4384158,4845349
6,Авиамоторная,Калининская линия,2024,11425579,11961284
13,Автозаводская,Замоскворецкая линия,2024,12952791,13584866
18,Автозаводская,Московское центральное кольцо,2024,1849625,2039005
23,Академическая,Калужско-Рижская линия,2024,11595003,11683305
...,...,...,...,...,...
1468,Юго-Западная,Сокольническая линия,2024,18541254,18819195
1473,Южная,Серпуховско-Тимирязевская линия,2024,8034606,7968476
1478,Ясенево,Калужско-Рижская линия,2024,10696850,10550298
1481,Яхромская,Люблинско-Дмитровская линия,2024,6312744,6162648


In [63]:
del station_year_passenger_flow['Line'], station_year_passenger_flow['Year']
station_year_passenger_flow

,NameOfStation,IncomingPassengers,OutgoingPassengers
1,Авиамоторная,4384158,4845349
6,Авиамоторная,11425579,11961284
13,Автозаводская,12952791,13584866
18,Автозаводская,1849625,2039005
23,Академическая,11595003,11683305
...,...,...,...
1468,Юго-Западная,18541254,18819195
1473,Южная,8034606,7968476
1478,Ясенево,10696850,10550298
1481,Яхромская,6312744,6162648


In [64]:
station_total_passenger_flow = station_year_passenger_flow.groupby('NameOfStation')[['IncomingPassengers', 'OutgoingPassengers']].sum().reset_index()
station_total_passenger_flow

,NameOfStation,IncomingPassengers,OutgoingPassengers
0,Авиамоторная,15809737,16806633
1,Автозаводская,14802416,15623871
2,Академическая,11595003,11683305
3,Александровский сад,6704109,6790020
4,Алексеевская,11082282,11310666
...,...,...,...
258,Юго-Западная,18541254,18819195
259,Южная,8034606,7968476
260,Ясенево,10696850,10550298
261,Яхромская,6312744,6162648


In [65]:
metro = pd.read_csv("moscow_metro_selenium.csv")

In [66]:
metro['station'] = metro['station'].str.lower()
metro = metro.drop_duplicates(subset="station")

In [67]:
station_total_passenger_flow['NameOfStation'] = station_total_passenger_flow['NameOfStation'].str.lower()

In [68]:
# Get unique station names from each DataFrame, converting to lowercase
metro_stations = set(metro['station'].dropna().unique())
metro_flow_stations = set(station_total_passenger_flow['NameOfStation'].dropna().unique())

# Find the symmetric difference
symmetric_diff = metro_stations.symmetric_difference(metro_flow_stations)

print(f"Total unique stations in metro (lowercase): {len(metro_stations)}")
print(f"Total unique stations in metro_flow (lowercase): {len(metro_flow_stations)}")
print(f"Number of stations in symmetric difference (not matched in either): {len(symmetric_diff)}")
print("\nStations that are not matched in both datasets (after lowercase conversion):")
for station_name in sorted(list(symmetric_diff)):
    print(f"- {station_name}")

Total unique stations in metro (lowercase): 239
Total unique stations in metro_flow (lowercase): 262
Number of stations in symmetric difference (not matched in either): 25

Stations that are not matched in both datasets (after lowercase conversion):
- андроновка
- балтийская
- белокаменная
- вавиловская
- верхние котлы
- выставочный центр
- зорге
- измайлово
- коптево
- лихоборы
- локомотив
- лужники
- новохохловская
- панфиловская
- площадь гагарина
- ростокино
- соколиная гора
- стрешнево
- телецентр
- угрешская
- улица академика королёва
- улица милашенкова
- улица сергея эйзенштейна
- хорошёво
- шелепиха


In [71]:
import requests
import pandas as pd
import time
import random
import logging

# === Настройки ===
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")

servers = [
    "https://overpass-api.de/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
    "https://overpass.nchc.org.tw/api/interpreter"
]

# === Функция запроса станции ===
def fetch_station_coords(station_name, max_servers=2):
    """
    Ищет координаты станции через Overpass.
    Пробует до max_servers разных серверов, по 3 попытки на каждый.
    Возвращает (lat, lon) или (None, None).
    """
    query = f"""
    [out:json][timeout:60];
    area["name"="Москва"]->.searchArea;
    (
      node["railway"~"station|halt|stop|stop_position|platform"]["name"~"{station_name}", i](area.searchArea);
      node["railway"~"station|halt|stop|stop_position|platform"]["name:ru"~"{station_name}", i](area.searchArea);
      node["public_transport"~"station|stop_position"]["name"~"{station_name}", i](area.searchArea);
    );
    out center;
    """

    for i, server in enumerate(random.sample(servers, len(servers))[:max_servers]):
        for attempt in range(3):
            try:
                response = requests.get(server, params={"data": query}, timeout=60)
                if response.status_code == 200 and response.text.strip():
                    result = response.json()
                    if result.get("elements"):
                        el = result["elements"][0]
                        return el["lat"], el["lon"]
                    else:
                        logging.warning(f"Не найдено на {server}: {station_name}")
                        break

                elif response.status_code in [429, 504]:
                    logging.warning(f"Ошибка {response.status_code} на {server}, повтор через 3 сек... ({station_name})")
                    time.sleep(random.uniform(2, 3))
                    continue

                else:
                    logging.warning(f"Ошибка {response.status_code} на {server}, пропуск ({station_name})")
                    break

            except Exception as e:
                logging.warning(f"Ошибка {e} на {server}, повтор через 3 сек...")
                time.sleep(random.uniform(2, 3))
                continue

        # после 3 неудачных попыток переходим на следующий сервер
        logging.warning(f"Смена сервера ({i+1}/{max_servers}) для {station_name}")
        time.sleep(random.uniform(3, 5))

    # если после max_servers ничего не найдено
    logging.warning(f"Пропускаю станцию после неудачных попыток: {station_name}")
    return None, None


# === Основной цикл ===
data = []
for name in symmetric_diff:
    name = name.title()
    logging.info(f"🔎 Поиск станции: {name}")
    lat, lon = fetch_station_coords(name)
    data.append({"station": name, "latitude": lat, "longitude": lon})
    time.sleep(random.uniform(3, 5))  # базовая пауза

df_coords = pd.DataFrame(data)
logging.info("Координаты собраны")

print(df_coords)


                     station   latitude  longitude
0                    Лужники  55.714373  37.566077
1                  Локомотив  55.803621  37.745960
2                      Зорге  55.777829  37.512073
3                 Балтийская  55.825836  37.496480
4             Соколиная Гора  55.771323  37.744973
5                Вавиловская  55.684916  37.540951
6          Улица Милашенкова  55.825523  37.583586
7                  Телецентр  55.821517  37.604162
8                   Хорошёво  55.776840  37.505660
9                    Коптево  55.841363  37.519358
10                  Лихоборы  55.842712  37.530476
11              Панфиловская  55.798999  37.498989
12         Выставочный Центр        NaN        NaN
13            Новохохловская  55.724122  37.717096
14                 Ростокино  55.841004  37.669905
15             Верхние Котлы  55.689697  37.622396
16              Белокаменная  55.829309  37.702484
17                 Стрешнево  55.810674  37.475078
18                Андроновка  5

In [72]:
df_coords_cleaned = df_coords.dropna(subset=['latitude', 'longitude'])
metro = pd.concat([metro, df_coords_cleaned]).drop_duplicates(subset='station', keep='first').reset_index(drop=True)
metro['station'] = metro['station'].str.lower()
metro

,station,latitude,longitude
0,бульвар рокоссовского,55.814800,37.734200
1,черкизовская,55.803800,37.744800
2,преображенская площадь,55.796300,37.715100
3,сокольники,55.788800,37.680200
4,красносельская,55.780100,37.667300
...,...,...,...
258,угрешская,55.715477,37.687445
259,улица академика королёва,55.821634,37.615863
260,улица сергея эйзенштейна,55.829641,37.649484
261,шелепиха,55.757300,37.525629


In [73]:
metro = pd.merge(metro, station_total_passenger_flow, left_on='station', right_on='NameOfStation', how='left')
metro = metro.drop(columns=['NameOfStation']) # Drop the redundant station name column from the merged dataframe
metro

,station,latitude,longitude,IncomingPassengers,OutgoingPassengers
0,бульвар рокоссовского,55.814800,37.734200,11923084.0,11973345.0
1,черкизовская,55.803800,37.744800,6768663.0,5918510.0
2,преображенская площадь,55.796300,37.715100,14787398.0,14953056.0
3,сокольники,55.788800,37.680200,12683076.0,13006092.0
4,красносельская,55.780100,37.667300,7035502.0,7207647.0
...,...,...,...,...,...
259,угрешская,55.715477,37.687445,1848391.0,1951249.0
260,улица академика королёва,55.821634,37.615863,62826.0,55920.0
261,улица сергея эйзенштейна,55.829641,37.649484,87429.0,76802.0
262,шелепиха,55.757300,37.525629,6152618.0,6367328.0


In [75]:
metro.to_csv('metro_stations_with_passenger_flow.csv', index=False)